## PointNet to Classify Proteins

### Dataset study

In [33]:
# Getting the data
import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import sys

home_path = '/home/giorgio/venvs/SHREC'
if home_path not in sys.path:
    sys.path.append(home_path)

from pandas import DataFrame
import pandas as pd

from pathlib import Path

from datasetStudy import *

debug_pointnet = False

# Multiple function here depends on this variable
#root = '/mnt/dataset/shrec-2025-protein-classification/v2-20250331' 
base_path = Path('/mnt/')
#base_path = Path('/mnt/raid1')
root = base_path / 'dataset/shrec-2025-protein-classification/v2-20250331'

#train_data = os.listdir(os.path.join(root, 'train'))
#train_data_cls = pd.read_csv('datasets/train_set-all.csv', sep=',', index_col=0)

train_data = os.listdir(root / 'train')
train_data_cls = pd.read_csv('../datasets/train_set-all.csv', sep=',', index_col=0)

print(train_data[:5])
print()
print(train_data_cls.info())
train_data_cls

['8ugd_8:R:3U_model1.vtk', '8h0v_18:R:c_model1.vtk', '3j3q_1:DX:4F_model1.vtk', '4u4u_23:XC:d1_model1.vtk', '6rny_4:H:H_model1.vtk']

<class 'pandas.core.frame.DataFrame'>
Index: 9244 entries, 0 to 9243
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   protein_id        9244 non-null   object
 1   class_id          9244 non-null   int64 
 2   number_of_points  9244 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 288.9+ KB
None


,protein_id,class_id,number_of_points
0,8ugd_8:R:3U_model1,96,5916
1,8h0v_18:R:c_model1,86,10078
2,3j3q_1:DX:4F_model1,8,18432
3,4u4u_23:XC:d1_model1,83,8242
4,6rny_4:H:H_model1,34,9204
...,...,...,...
9239,3j3y_1:HL:6R_model1,8,18342
9240,4u4y_15:O:C3_model1,91,12976
9241,7w31_26:AA:c_model1,18,16454
9242,3j4k_1:E:E_model1,90,23188


In [34]:
src = root / 'train-xz'
mountpoint = '/tmp/ramdrive'
size_in_gb = 3

def create_ramdrive(src, mountpoint, size_in_gb):
    if Path(mountpoint).exists() or Path(mountpoint).is_dir():
        print(f'Ramdrive already created, exiting...')
        return
    
    !sudo mkdir -p "{mountpoint}"
    !sudo mount -o size="{size_in_gb}"G -t tmpfs none "{mountpoint}"
    !rsync -a --progress "{src}" "{mountpoint}"
    
def unmount_ramdrive(mountpoint):
    !sudo umount "{mountpoint}"
    !sudo rmdir "{mountpoint}"

In [35]:
create_ramdrive(src, mountpoint, size_in_gb)
#unmount_ramdrive(mountpoint)

Ramdrive already created, exiting...


In [36]:
possible_disconnected_mesh(train_data_cls, 96, False)

disconnected_dict = {}
for idx in range(97):
    disconnected_dict[idx] = possible_disconnected_mesh(train_data_cls, idx)

#print(disconnected_dict)

_, damaged = possible_disconnected_mesh(train_data_cls, 8, True); damaged

,protein_id,class_id,number_of_points


In [37]:
dist = cls_distribution(train_data_cls)
inspect_distribution(dist, l_lim=0, u_lim=5)

Classes which have between 0 and 5 element: 16/97


#### Create and save new dataframe

In [38]:
dist_all = cls_distribution(train_data_cls)

# Filter the number of points
raw_train_dataframe_f1 = number_of_point_filter(train_data_cls, 1000)
dist_f1 = cls_distribution(raw_train_dataframe_f1)


# Filter according to the number of samples in classes
raw_train_dataframe_f2 = number_of_class_filter(raw_train_dataframe_f1, l_cut_off=10)
dist_f2 = cls_distribution(raw_train_dataframe_f2)

raw_train_dataframe_f3 = number_of_class_filter(raw_train_dataframe_f1, l_cut_off=10, u_cut_off=100)
dist_f3 = cls_distribution(raw_train_dataframe_f3)

raw_train_dataframe_f4 = number_of_class_filter(raw_train_dataframe_f1, l_cut_off=10, u_cut_off=50)
dist_f4 = cls_distribution(raw_train_dataframe_f4)

raw_train_dataframe_f5 = number_of_class_filter(raw_train_dataframe_f1, l_cut_off=100)
dist_f5 = cls_distribution(raw_train_dataframe_f5)

raw_train_dataframe_f6 = number_of_class_filter(raw_train_dataframe_f1, l_cut_off=0, u_cut_off=450)
dist_f6 = cls_distribution(raw_train_dataframe_f6)

distributions = [dist_all, dist_f1, dist_f2, dist_f3, dist_f4, dist_f5, dist_f6]
for idx in range(len(dist_all)):
    output = f'cls {idx}: '
    for dist in distributions:
        output += f'{print_dist(dist, idx)}\t'
    print(output)

print(f'{len(train_data_cls) = }')
print(f'{len(raw_train_dataframe_f1) = }')
print(f'{len(raw_train_dataframe_f2) = }')
print(f'{len(raw_train_dataframe_f3) = }')
print(f'{len(raw_train_dataframe_f4) = }')
print(f'{len(raw_train_dataframe_f5) = }')

cls 0: 101	101	101	0	0	101	101	
cls 1: 3	3	0	0	0	0	3	
cls 2: 17	17	17	17	17	0	17	
cls 3: 21	21	21	21	21	0	21	
cls 4: 15	15	15	15	15	0	15	
cls 5: 127	127	127	0	0	127	127	
cls 6: 25	25	25	25	25	0	25	
cls 7: 75	75	75	75	0	0	75	
cls 8: 2054	2054	2054	0	0	2054	0	
cls 9: 66	66	66	66	0	0	66	
cls 10: 14	14	14	14	14	0	14	
cls 11: 43	43	43	43	43	0	43	
cls 12: 10	10	10	10	10	0	10	
cls 13: 22	22	22	22	22	0	22	
cls 14: 489	489	489	0	0	489	0	
cls 15: 89	89	89	89	0	0	89	
cls 16: 154	154	154	0	0	154	154	
cls 17: 116	116	116	0	0	116	116	
cls 18: 76	76	76	76	0	0	76	
cls 19: 42	42	42	42	42	0	42	
cls 20: 7	7	0	0	0	0	7	
cls 21: 74	74	74	74	0	0	74	
cls 22: 58	58	58	58	0	0	58	
cls 23: 10	10	10	10	10	0	10	
cls 24: 10	10	10	10	10	0	10	
cls 25: 143	143	143	0	0	143	143	
cls 26: 2	2	0	0	0	0	2	
cls 27: 3	3	0	0	0	0	3	
cls 28: 18	18	18	18	18	0	18	
cls 29: 14	14	14	14	14	0	14	
cls 30: 3	3	0	0	0	0	3	
cls 31: 33	33	33	33	33	0	33	
cls 32: 93	93	93	93	0	0	93	
cls 33: 126	126	126	0	0	126	126	
cls 34: 73	73	73	73	0	0	73	
c

In [39]:
cls_selected = [8, 90, 56, 14, 61, 70, 86, 5, 16, 25, 54, 62, 74, 83, 91, 92, 17, 45, 87, 71]

df_3 = create_dataframe(raw_train_dataframe_f1, class_ids=cls_selected[:2],  number_of_proteins=500)
df_4 = create_dataframe(raw_train_dataframe_f1, class_ids=cls_selected[:10], number_of_proteins=100)
df_5 = create_dataframe(raw_train_dataframe_f1, class_ids=cls_selected[:20], number_of_proteins=50)
df_6 = create_dataframe(raw_train_dataframe_f1, class_ids=cls_selected[:20], number_of_proteins=100)
#df_7 = create_dataframe(raw_train_dataframe_f6, class_ids=dist_f6_available_classes, number_of_proteins=100000)
df_8 = create_dataframe(raw_train_dataframe_f1, class_ids=[40, 88], number_of_proteins=10000)
df_9 = create_dataframe(raw_train_dataframe_f1, class_ids=[0, 1], number_of_proteins=1000)
df_10 = create_dataframe(raw_train_dataframe_f1, class_ids=[8,14], number_of_proteins=3000)
'''
df_3.to_csv('datasets/train_set-2_cls-1000_images.csv')
df_4.to_csv('datasets/train_set-10_cls-1000_images.csv')
df_5.to_csv('datasets/train_set-20_cls-1000_images.csv')
df_6.to_csv('datasets/train_set-20_cls-2000_images.csv')
df_7.to_csv('datasets/train_set-all-cls_except-cls-larger-than-450.csv')
symmetria_df.to_csv('datasets/train_set-symmetria-100k-easy-astroid-citrus-10k-samples.csv')
df_8.to_csv('datasets/train_set-2_cls_40_88-192_proteins.csv')
df_9.to_csv('../datasets/train_set_unbalanced-2_cls_0_1.csv')
df_10.to_csv('../datasets/train_set_unbalanced-2_cls_8_14.csv')
'''

raw_train_dataframe_f1.to_csv('../datasets/train_set_comprehensive.csv')



#### Visulize protein

In [40]:
cls_id = 1
df = raw_train_dataframe_f1[raw_train_dataframe_f1['class_id'] == cls_id]
print(len(df))
visualize_mesh(raw_train_dataframe_f1, cls=cls_id, idx=3)

3
PolyData (0x70b7581f9ba0)
  N Cells:    11576
  N Points:   5788
  N Strips:   0
  X Bounds:   -1.384e+01, 1.552e+01
  Y Bounds:   -1.553e+01, 1.623e+01
  Z Bounds:   -1.842e+01, 2.068e+01
  N Arrays:   3


Widget(value='<iframe src="http://localhost:32931/index.html?ui=P_0x70b6f6be4200_1&reconnect=auto" class="pyvi…

### Augmentations

In [41]:
from format import Text
import torch
from symmetria.transformations import *
from symmetria.shapes import BenchmarkShape

In [42]:
points_rectangle = np.array([[0, 2, 0, 0, 2, 2, 0, 2],
                             [0, 0, 1, 0, 1, 0, 1, 1],
                             [0, 0, 0, 1, 0, 1, 1, 1]]).astype(np.float32)

points_rectangle_t = torch.from_numpy(points_rectangle)

rot = random_rotation_matrix(rotate_x=False, rotate_z=False)

shapebench = BenchmarkShape(points_rectangle_t)
#shapebench.apply_rotation(torch.from_numpy(rot))
#shapebench.apply_traslation(1,1,0)
#shapebench.apply_uniform_noise(100, 1)
#shapebench.apply_gaussian_noise(100, 1)

rect = pv.PolyData(shapebench.points.numpy().T)
#rect.plot()

In [43]:
from random import random

class RotateAroundZero():
    def __init__(self, p=0.5, rot=None):
        self.p = p
        self.rot = rot

    def __call__(self, points):
        if random() < self.p:
            if self.rot == None:
                rot = torch.from_numpy(random_rotation_matrix())
            else:
                rot = self.rot
        
            rot = rot.to(torch.device(points.device))

            ones = torch.ones((1,points.shape[0]), device=points.device)
            coords = torch.concatenate((torch.transpose(points, 0, 1), ones))
            
            coords = rot@coords
            
            points = torch.transpose(coords[:3,:], 0, 1)

        return points
    
    def __repr__(self):
        return f'RotateAroundZero(p={repr(self.p)}, rot={repr(self.rot)})'
    
class Translate():
    def __init__(self, p=0.5, shift=None, scale=1):
        self.p = p
        self.shift = shift
        self.scale = scale

    def __call__(self, points):        
        if random() < self.p:
            if self.shift == None:
                shift = (torch.rand(1, device=points.device) * self.scale,
                         torch.rand(1, device=points.device) * self.scale,
                         torch.rand(1, device=points.device) * self.scale)
            else:
                shift = self.shift
            
            for i in range(len(shift)):
                points[:,i] += shift[i]
                   
        return points
    
    def __repr__(self):
        return f'Translate(p={repr(self.p)}, shift={repr(self.shift)}, scale={repr(self.scale)})'
    
class UniformNoise():
    def __init__(self, n, T, p=0.5):
        self.p = p
        self.n, self.T = n, T
    
    def __call__(self, points):
        if random() < self.p:
            num_points = points.shape[0]

            indices = torch.randperm(num_points, device=points.device)[:self.T]

            points[indices,:] = points[indices,:] + (2*torch.rand(self.T, 3, device=points.device)-1)/self.n
        
        return points
    
    def __repr__(self):
        return f'UniformNoise(n={repr(self.n)}, T={repr(self.T)}, p={repr(self.p)})'
    
class GaussianNoise():
    def __init__(self, n, T, p=0.5):
        self.p = p
        self.n, self.T = n, T
    
    def __call__(self, points):
        if random() < self.p:
            num_points = points.shape[0]

            indices = torch.randperm(num_points, device=points.device)[:self.T]
    
            points[indices,:] = points[indices,:] + torch.rand(self.T, 3, device=points.device)/self.n

        return points
    
    def __repr__(self):
        return f'GaussianNoise(n={repr(self.n)}, T={repr(self.T)}, p={repr(self.p)})'

In [44]:
points_rectangle = np.array([[0, 2, 0, 0, 2, 2, 0, 2],
                             [0, 0, 1, 0, 1, 0, 1, 1],
                             [0, 0, 0, 1, 0, 1, 1, 1]]).astype(np.float32).T

points_rectangle_t = torch.from_numpy(points_rectangle).to(torch.device('cuda:0'))

translate = Translate(p=1)
print(translate)
points_rectangle_t = translate(points_rectangle_t)
print(translate)
rect = pv.PolyData(points_rectangle)
#rect.plot()

rect_tfm = pv.PolyData(points_rectangle_t.cpu().numpy())
#rect_tfm.plot()

Translate(p=1, shift=None, scale=1)
Translate(p=1, shift=None, scale=1)


### DataSet and DataLoader

#### Dataset

In [46]:
import numpy as np
import lzma
from torch.utils.data import Dataset, DataLoader
from symmetria.transforms.RandomSampler import RandomSampler
from symmetria.transforms.UnitSphereNormalization import UnitSphereNormalization
from tqdm import tqdm

class ProteinDataset(Dataset):
    def __init__(self, data_df, tfms, balancing='None', root=root, extention='xz', train=True):
        super().__init__()
        self.df = data_df
        self.tfms = tfms
        self.balancing = balancing
        self.extention = extention
        
        self.train = train
        if self.train:
            if self.extention == 'vtk':
                self.root = os.path.join(root, 'train')

            elif self.extention == 'xz':
                #self.root = os.path.join(root, 'train-symmetria-xz') # TODO roll back
                self.root = os.path.join(root, 'train-xz')
            else:
                raise UserWarning('Extention not supported')
            
        else:
            raise UserWarning('Not yet implemented for the test dataset')
        
        self.encode_label()
        self.loader()
    
    def loader(self):
        self.data = []
        
        # Defining the balancing factor for each class
        self.set_balancing()

        # Loading data
        for index in tqdm(self.df.index):
            protein, cls, nop = self.df['protein_id'].loc[index], self.df['class_id'].loc[index], self.df['number_of_points'].loc[index]

            cls_t = torch.tensor(self.encoded_cls[cls]).to(torch.device('cuda:0'))

            if self.extention == 'vtk':
                point_cloud = self.get_vtk_points(protein)

            elif self.extention == 'xz':
                point_cloud = self.get_xz_points(protein, cls, nop)

            point_cloud_t = torch.from_numpy(point_cloud).to(torch.device('cuda:0'))     

            point_cloud_t, cls_t = point_cloud_t.type(torch.float32), cls_t.type(torch.float32)
            
            for _ in range(self.balancing_factors[cls]):
                self.data.append((point_cloud_t, cls_t))

    def get_vtk_points(self, name):
        prot_file = name + '.' + self.extention
        prot_file = os.path.join(self.root, prot_file)
        
        prot_mesh = pv.read(prot_file)

        return prot_mesh.points

    def get_xz_points(self, name, cls, nop=None, debug=False):
        cls = str(cls)
        nop = str(nop) if nop and nop != 'nan' else None

        while len(cls) < 2:
            cls = '0' + cls
        
        if nop and nop != 'nan':
            while len(nop) < 6:
                nop = '0' + nop

        if nop and nop != 'nan':
            prot_file = cls + '-' + nop +  '-' + name.replace(':', '+') + '.' + self.extention
        else:
            prot_file = cls + '-' + name.replace(':', '+') + '.' + self.extention
        if debug:
            print(f'ProteinDataset is loading files from {self.root}')
        prot_file = os.path.join(self.root, prot_file)        
        
        with lzma.open(prot_file, 'rt') as f:
            point_cloud = np.loadtxt(f)

        return point_cloud

    def set_balancing(self):
        self.balancing_factors = {}
        dist = cls_distribution(self.df)
        for key, value in dist.items():
            if self.balancing == 'Proportional':
                self.balancing_factors[key] = max_of_dist(dist)//value
            
            elif self.balancing == 'Weak':
                factor = max_of_dist(dist)//value

                if factor < 10:
                    self.balancing_factors[key] = factor
                else:
                    self.balancing_factors[key] = int(10)

            elif self.balancing == 'None':
                self.balancing_factors[key] = int(1)
            
            else:
                raise(UserWarning(f'{self.balancing} is an invalid option'))

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index, do_transform=True):
        prot, cls = self.data[index]
        
        if do_transform:
            for tfm in self.tfms:
                prot = tfm(prot)

        # NOTE: here we transpose the points (from Nx3 to 3xN) to then pass them to the PoinNet encoder
        prot = torch.transpose(prot, 0, 1)
        return (prot, cls)
    
    def encode_label(self):
        self.encoded_cls = {}
        
        prot_clss = np.sort(self.df['class_id'].unique())
        
        #One hot encoding
        '''
        for idx, cls in enumerate(prot_clss):
            self.encoded_cls[cls] = np.eye(len(prot_clss))[idx]
        '''

        for idx, cls in enumerate(prot_clss):
            self.encoded_cls[cls] = int(idx)

    def render_pointcloud(self, index):
        prot, _ = self.data[index]
        prot = torch.transpose(prot, 0, 1).numpy()
        cloud = pv.PolyData(prot)
        print(cloud)
        cloud.plot()
    

transforms = [UnitSphereNormalization(),
              Translate(p=0.8, scale=1),
              RotateAroundZero(p=0.8),
              GaussianNoise(n=100, T=2500, p=0.8),
              RandomSampler(sample_size=5000)]


def generate_train_valid_set(df, tfms, val_pct, root=root, seed=42, **kwargs):
    '''
    Can also take some other arguments to be passed to the dataset initializer

            -> path (str): path to the parent directory containing the train files 
    '''

    train_size = int(len(df) * (1 - val_pct))    
    df_train = df.sample(train_size, random_state=seed)
    df_valid = df.drop(df_train.index)

    return ProteinDataset(df_train, tfms, root=root, **kwargs), ProteinDataset(df_valid, tfms, root=root, **kwargs)

'''
def generate_train_valid_set_boring(df, val_pct, seed=42):
    train_size = int(len(df) * val_pct)    
    df_train = df.sample(train_size, random_state=seed)
    df_valid = df.drop(df_train.index)

    return df_train, df_valid
'''

'\ndef generate_train_valid_set_boring(df, val_pct, seed=42):\n    train_size = int(len(df) * val_pct)    \n    df_train = df.sample(train_size, random_state=seed)\n    df_valid = df.drop(df_train.index)\n\n    return df_train, df_valid\n'

#### Testing Dataset

In [ ]:
example_dataframe = create_dataframe(raw_train_dataframe_f1, class_ids=[2, 5, 30], number_of_proteins=100000)
example_set_train, example_set_valid = generate_train_valid_set(example_dataframe, transforms, 0.2, balancing=True)

In [ ]:
print(example_set_train.balancing_factors)
print(example_set_train.encoded_cls)
cls_distribution(example_set_train.df)

In [ ]:
example_set_train.__getitem__(0, do_transform=False)

In [ ]:
prot = example_set_train.__getitem__(0, do_transform=False)

In [ ]:
print(f'{Text(prot, 'prot'):content}')

In [ ]:
print(f'{Text(prot[0], 'prot[0]'):content}')

In [ ]:
print(f'{prot[0].min() = } - {prot[0].max() = }')

In [ ]:
unit_normalizer = UnitSphereNormalization()
prot, _ = example_set_train.__getitem__(0, do_transform=False)
prot = torch.transpose(prot, 0, 1)
norm_prot = unit_normalizer(prot)

In [ ]:
print(f'{Text(norm_prot, 'norm_prot'):content}')

In [ ]:
print(f'{norm_prot.min() = } - {norm_prot.max() = }')

In [ ]:
print(f'{unit_normalizer.centroid.min() = } - {unit_normalizer.centroid.max() = }')

In [ ]:
unit_normalizer.centroid.shape

In [ ]:
points = prot[0]

In [ ]:
points

### PointNet class

In [ ]:
import importlib
import symmetria
importlib.reload(symmetria)
importlib.reload(symmetria.decoders)
importlib.reload(symmetria.decoders.prediction_head)

#### PointNet

In [47]:
import torch.nn as nn
import torch.nn.functional as F
from symmetria.encoders.pointnet_encoder import PointNetEncoder

from symmetria.encoders.pointnext.pointnext_encoder_parameters import *
from symmetria.encoders.pointnext.pointnext_encoder import PointNeXt

from symmetria.decoders.prediction_head import PredictionHead

if debug_pointnet:
    bs, sz = 1, 2048
    encoder = PointNetEncoder(use_bn=False)
    mock_x = torch.randn(bs, 3, sz)
    output = encoder.forward(mock_x)
    print(f'{Text(output, 'output'):inspect}')

    decoder = PredictionHead(1024, 96)
    output_decoder = decoder.forward(output)
    print(f'{Text(output_decoder, 'output_decoder'):inspect}')

class PointNet(nn.Module):
    def __init__(self, output_size, max_points, use_bn=False, encoder='pointnet'): # make it prettier like in segmenter
        super().__init__()
        
        if encoder in POINTNEXT_MODEL_CONFIG:
            model_cfg = POINTNEXT_MODEL_CONFIG[encoder]     # 'PointNeXt_B' (21.5 M), 'PointNeXt_L2' (32.0 M), 'PointNeXt_XXL' (73.8 M)
            self.encoder = PointNeXt(model_cfg)
            self.encoder_output_size = output_size          # because of the adapter head (e.g. 2048 -> 1024) within the encoder
            print(f"Using PointNeXt {model_cfg} - encoder: {self.encoder}")
        else:
            self.encoder = PointNetEncoder(use_bn)
            print(f"Using PointNet as encoder - encoder: {self.encoder}")
        
        self.max_points = max_points

        self.input_size = self.get_input_size()        
        self.output_size = output_size

        print(f'Creating a PredictionHead with {self.input_size} as input and {self.output_size} as output')
        self.decoder = PredictionHead(self.input_size, self.output_size, use_bn, use_relu=True)
        
        #self.softmax = nn.Softmax(dim=1)

    def get_input_size(self):
        mock_x = torch.randn(1, 3, self.max_points)
        return self.encoder(mock_x).shape[-1]

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        #x = self.softmax(x)

        return x #SoftMax already inside the CrossEntropyLoss

### Config Definition

In [48]:
from datetime import datetime
from torch import optim

time_now = datetime.now()
time_now = time_now.strftime('%d%m%Y_%H%M%S')
_folder = !pwd

env = {}

### Debug
env['debug_loss'] = False

env['project_folder'] = _folder[0]
env['project'] = 'shrec-2025'
env['run'] = env['project'] + '_' + time_now
#env['output_dir'] = os.path.join(env['project_folder'], 'wandb', 'run_' + env['run'])
if 'output' not in os.listdir(env['project_folder']):
    os.mkdir('output')
env['output_dir'] = Path(env['project_folder']) / 'output'

env['max_points'] = 4000

#env['dataset'] = 'train_set-2_cls-1000_images.csv' 
#env['dataset'] = 'train_set_comprehensive.csv'
env['dataset'] = 'train_set-all.csv'
#env['dataset'] = 'train_set-all-cls_except-cls-larger-than-450.csv'
#env['dataset'] = 'train_set-symmetria-100k-easy-astroid-citrus-10k-samples.csv'
#env['dataset'] = 'train_set-2_cls-1000_images.csv'
#env['dataset'] = 'train_set-10_cls-1000_images.csv'
#env['dataset'] = 'train_set_unbalanced-2_cls_0_1.csv'
#env['dataset'] = 'train_set_unbalanced-2_cls_8_14.csv'

#env['dataset_path'] = '/mnt/dataset/shrec-2025-protein-classification/v2-20250331'
env['dataset_path'] = root

'''
Balancing options:
    'Proportional'
        the balancing fatcor for each class is computed with respect to the class with more elements
    'Weak'
        same as 'Proportional' but with a cut off of 10 over the balancing factor
    'None'
        balancing factor fixed on 1 
'''
env['dataset_balancing'] = 'None'

ds_path = Path('../datasets') / env['dataset']
print(f'Creating a test dataset from {ds_path}')

test = pd.read_csv(ds_path, index_col=0)
print(f'Test dataset has {len(test)} rows')

Creating a test dataset from ../datasets/train_set-all.csv
Test dataset has 9244 rows


### Model Definition

In [ ]:
env['device'] = 'cuda:0'
env['val_pct'] = 0.2
env['augmentations_on'] = True
'''
env['aug'] = [Translate(p=0.8),
              UnitSphereNormalization(),
              RotateAroundZero(p=0.8),
              GaussianNoise(n=10, T=(env['max_points']//2),  p=0.8),
              RandomSampler(sample_size=env['max_points'])]
'''

env['aug'] = [Translate(p=0.5),
              UnitSphereNormalization(),
              RandomSampler(sample_size=env['max_points']),
              GaussianNoise(n=10, T=(env['max_points']//2),  p=0.5),
              ]

#env['aug'] = [UnitSphereNormalization()]
#env['aug'] = []

env['bs'] = 2
env['epochs'] = 100
env['lr'] = 1e-4
env['momentum'] = 0
#env['wd'] = 5e-2 # because "PointNeXt is trained with a weight decay of 0.05 for 250 epochs" - nope, too large, doesn't learn!
env['wd'] = 1e-3 # because of this: https://github.com/yanx27/Pointnet_Pointnet2_pytorch/blob/master/train_classification.py

env['encoder']   = 'pointnet'
#env['encoder']  = 'PointNeXt_XXL'
env['model']     = PointNet(output_size=len(test['class_id'].unique()), max_points=env['max_points'], encoder=env['encoder']).to(torch.device(env['device']))

env['model_pth'] = '/tmp/pointnet.pth'
env['pretrain']  = True

if env['pretrain']:
    if not Path(env['model_pth']).exists():
        !wget 'https://github.com/meder411/PointNet-PyTorch/raw/refs/heads/master/classifier_model_state.pth' --output-document "{env['model_pth']}"
    else:
        print(f'{env['model_pth']} already exists, skipping download...')
    
    print(f'Loading file {env['model_pth']}')
    env['model'].load_state_dict(torch.load(env['model_pth'], map_location=torch.device(env['device'])), strict=False)

env['epoch_checkpoints'] = 20
env['save_folder'] = env['output_dir']/Path(f'run_{env['run']}_{env['dataset'].split('.')[0]}')
print(f'Saving models checkpoints to {env['save_folder']}')

env['loss_func'] = nn.CrossEntropyLoss()
env['optimizer'] = optim.Adam(env['model'].parameters(), lr=env['lr'], weight_decay=env['wd'])
'''
env['optimizer'] = optim.SGD(env['model'].parameters(),
                             lr=env['lr'],
                             momentum=env['momentum'],
                             weight_decay=env['wd'])
'''

Using PointNet as encoder - encoder: PointNetEncoder(
  (input_transform): TNet(
    (shared_mlps): Sequential(
      (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (1): ReLU()
      (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (3): ReLU()
      (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (5): ReLU()
    )
    (linear): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=9, bias=True)
    )
  )
  (feature_transform): TNet(
    (shared_mlps): Sequential(
      (0): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (1): ReLU()
      (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (3): ReLU()
      (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (5): ReLU()
    )
    (linear): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
     

[W410 09:00:18.825657960 NNPACK.cpp:62] Could not initialize NNPACK! Reason: Unsupported hardware.


"\nenv['optimizer'] = optim.SGD(env['model'].parameters(),\n                             lr=env['lr'],\n                             momentum=env['momentum'],\n                             weight_decay=env['wd'])\n"

In [ ]:
env

{'debug_loss': False,
 'project_folder': '/home/giorgio/venvs/SHREC/notebooks',
 'project': 'shrec-2025',
 'run': 'shrec-2025_10042025_090017',
 'output_dir': PosixPath('/home/giorgio/venvs/SHREC/notebooks/output'),
 'max_points': 4000,
 'dataset': 'train_set-all.csv',
 'dataset_path': PosixPath('/mnt/dataset/shrec-2025-protein-classification/v2-20250331'),
 'dataset_balancing': 'None',
 'device': 'cuda:0',
 'val_pct': 0.2,
 'augmentations_on': True,
 'aug': [Translate(p=0.5, shift=None, scale=1),
  GaussianNoise(n=10, T=2000, p=0.5)],
 'bs': 2,
 'epochs': 100,
 'lr': 0.0001,
 'momentum': 0,
 'wd': 0.001,
 'encoder': 'pointnet',
 'model': PointNet(
   (encoder): PointNetEncoder(
     (input_transform): TNet(
       (shared_mlps): Sequential(
         (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
         (1): ReLU()
         (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
         (3): ReLU()
         (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
         (5): ReLU()
  

### Training Loop

In [ ]:
import wandb
from sklearn.metrics import confusion_matrix

def calculate_loss(logits, target, loss_func):
    output    = torch.argmax(logits, dim=1)
    target    = target.to(torch.int64)
    #softmaxed = torch.softmax(logits, dim=1)
    loss      = loss_func(logits, target)
    return loss, output, target

def process_confusion_matrix(matrix):
    weights = np.zeros(len(matrix))
    cls_acc = np.zeros(len(matrix))
    for i in range(len(matrix)):
        row_sum = 0
        for j in range(len(matrix)):
            row_sum += matrix[i][j]
        
        weights[i] = row_sum
        cls_acc[i] = matrix[i][i]/row_sum

    total_acc = np.sum(cls_acc * weights)/np.sum(weights)

    return total_acc, cls_acc

def train(train_dl, valid_dl, env):
    net = env['model']
    loss_func = env['loss_func']
    optimizer = env['optimizer']

    best_val_loss = 1.
    wandb.init(project=env['project'], name=env['run'], dir=env['output_dir'], config=env)

    for epoch in range(env['epochs']):
        print('/' * 20 + f' Epoch: {epoch + 1} ' + '/' * 20)
        for step, batch in enumerate(tqdm(train_dl)):
            input_p, target = batch

            net.zero_grad()

            logits = net(input_p)
            #print(f'{Text(output, 'output'):content}')
            #print(f'{Text(target, 'target'):content}')
            
            '''
            output = torch.argmax(logits, dim=1)
            target = target.to(torch.int64)
            loss = loss_func(output, target)
            '''
            loss, output, target = calculate_loss(logits, target, loss_func)
            if env['debug_loss'] and step % 1000 == 0:
                print(f'pred: {int(output)} - target: {int(target)} - loss: {loss:.3f} - raw pred: {logits}')
            wandb.log({'epoch': epoch,
                    'loss': loss})
            #print(f'Training loss: {loss}')
            loss.backward()
            optimizer.step()
        
        if not env['save_folder'].exists():
            env['save_folder'].mkdir()

        if epoch % env['epoch_checkpoints'] == 0 and epoch != 0:
            save_path = env['save_folder']/Path(f'model_checkpoint_epoch-{epoch}.pth')
            torch.save(net.state_dict(), save_path)
        
        print('/' * 20 + ' Validation ' + '/' * 20)
        with torch.no_grad():
            val_loss = []
            acc = []
            conf_matrix = np.zeros(shape=(net.output_size, net.output_size), dtype=np.int32)
            for step, batch in enumerate(tqdm(valid_dl)):
                input_p, target = batch
                
                logits = net(input_p)

                '''
                output = torch.argmax(logits, dim=1)
                target = target.to(torch.int64)
                loss = loss_func(output, target).cpu()
                '''
                loss, output, target = calculate_loss(logits, target, loss_func)
                if env['debug_loss'] and step % 1000 == 0:
                    print(f'pred: {int(output)} - target: {int(target)} - loss: {loss:.3f} - raw pred: {logits}')
                
                val_loss.append(loss.cpu())

                # Mean accuracy
                #matching = [torch.argmax(i) == torch.argmax(j) for i, j in zip(output, target)]
                matching = [int(i) == int(j) for i, j in zip(output, target)]
                accuracy = matching.count(True)/len(matching)

                if env['debug_loss'] and step % 10 == 0:
                    print(f'{matching = } - {accuracy = }')
                acc.append(accuracy)

                confusion_matrix_batch = np.zeros(shape=(net.output_size, net.output_size), dtype=np.int32)
                for targ, pred in zip(target.cpu().numpy(), output.cpu().numpy()):
                    confusion_matrix_batch[targ][pred] += 1
            
                conf_matrix += confusion_matrix_batch
            
            print(f'{Text(conf_matrix, 'conf_matrix'):content}')
            total_acc, cls_acc = process_confusion_matrix(conf_matrix)
            
            mean_val_loss = np.mean(val_loss)
            if mean_val_loss < best_val_loss:
                best_val_loss = mean_val_loss
                save_best_path = env['save_folder']/Path('model_best_loss.pth')
                torch.save(net.state_dict(), save_best_path)

            log = {'epoch': epoch,
                   'step': step,
                   'val_loss': mean_val_loss,
                   'accuracy': np.mean(acc),
                   'total_acc': total_acc}
            
            for cls, val in enumerate(cls_acc):
                key = f'class_{cls}_acc'
                log[key] = val
            
            wandb.log(log)

    save_final_model = env['save_folder']/'model_final.pth'
    torch.save(net.state_dict(), save_final_model)
    
    wandb.finish()

In [20]:

test_train_ds, test_valid_ds = generate_train_valid_set(df=test,
                                                        tfms=env['aug'],
                                                        val_pct=env['val_pct'],
                                                        root=mountpoint,
                                                        balancing=env['dataset_balancing'])

'''
def validation_set(df, valid_pct):
    subset_dfs = []
    class_ids = df['class_id'].unique()

    for cls in class_ids:
        class_df = df[df['class_id'] == cls]
        if len(class_df) * valid_pct < 1:
            subset = class_df.sample(n=1)
        else:
            subset = class_df.sample(frac=valid_pct)

        subset_dfs.append(subset)
    
    dataset = pd.concat(subset_dfs)
    return dataset.sample(frac=1, ignore_index=True)

test_train_ds = ProteinDataset(test, tfms=env['aug'], root=mountpoint, balancing=env['dataset_balancing'])
test_valid_ds = ProteinDataset(validation_set(test, 0.3), tfms=env['aug'], root=mountpoint, balancing=env['dataset_balancing'])
'''
test_train_dl, test_valid_dl = DataLoader(test_train_ds, batch_size=env['bs'], shuffle=True), DataLoader(test_valid_ds, batch_size=env['bs'], shuffle=True)

NameError: name 'generate_train_valid_set' is not defined

In [ ]:
len(test_valid_ds.df['class_id'].unique())

97

In [ ]:
print(f'{len(test_train_ds) = }\n{len(test_valid_ds) = }')

len(test_train_ds) = 59435
len(test_valid_ds) = 17853


In [ ]:
train(test_train_dl, test_valid_dl, env)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: g-palmieri4998 (g-palmieri4998-cnr-imati) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


//////////////////// Epoch: 1 ////////////////////


100%|██████████| 3698/3698 [00:28<00:00, 129.61it/s]


//////////////////// Validation ////////////////////


100%|██████████| 925/925 [00:03<00:00, 271.99it/s]
/tmp/ipykernel_395505/34065137.py:20: RuntimeWarning: invalid value encountered in scalar divide
  cls_acc[i] = matrix[i][i]/row_sum


...

//////////////////// Validation ////////////////////


100%|██████████| 925/925 [00:02<00:00, 429.42it/s]

conf_matrix type = <class 'numpy.ndarray'>
conf_matrix device = cpu
conf_matrix dtype = int32
conf_matrix shape = (97, 97)
conf_matrix = [[19  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  5 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]



accuracy,▁▁▁▁▂▂▂▃▂▂▃▄▃▅▅▆▆▆▆▆▇▇▆▇▇▇█▇▇▇▇▇▆▇▇████▇
class_0_acc,▁▁▁▁▁▁▁▁▁▁▂▃▆▆▇▇██████▇██▄█▇█████████▆██
class_10_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▁█▅▅██▁█▅████
class_11_acc,▁▁▁▁▁▁▁▁▁▁▄▃▅▃▅▄▅▇▅▆▇█▆▇█▇▆█▆█▇█▅██▇▆▇██
class_12_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▃▃▃▃▃▆▆▆▆▆█▆▆▆▆▆▆▆▆▆▆▆▆▆
class_13_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
class_14_acc,▁▁▁▃▃▅▆▆▅▆▅▇▇▄▆▆▅▇▆▇▄▆▆▄▇▄▇▆▇▇▆▅▆▆▅█▇▇█▇
class_15_acc,▁▁▁▁▂▁▃▂█▆▇█▆█▇██▇████▇██▄█▇█▇█▆███▂████
class_16_acc,▁▁▁▁▁▁▁▁▁▃▅▅▃▅▅▆▇█▇▆█▇████████▇▇██▇█▆██▇
class_17_acc,▁▁▁▁▁▁▁▁▁▁▃▅▆▆▆▆▇▇█▇█▇▇▇█▇██▅▇▇▇█▇▇█▇█▇█
class_18_acc,▁▁▁▁▁▆▇▇▇▇▇▆▆▇▆▆▆▆▇▇▇▇▇█▇▇▇▇▇██▇▇▇██▇█▇▇


message_loop has been closed
Traceback (most recent call last):
  File "/home/giorgio/venvs/SHREC/lib/python3.12/site-packages/wandb/sdk/interface/router_sock.py", line 27, in _read_message
    return self._sock_client.read_server_response(timeout=1)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/giorgio/venvs/SHREC/lib/python3.12/site-packages/wandb/sdk/lib/sock_client.py", line 235, in read_server_response
    data = self._read_packet_bytes(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/giorgio/venvs/SHREC/lib/python3.12/site-packages/wandb/sdk/lib/sock_client.py", line 220, in _read_packet_bytes
    raise SockClientClosedError
wandb.sdk.lib.sock_client.SockClientClosedError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/giorgio/venvs/SHREC/lib/python3.12/site-packages/wandb/sdk/interface/router.py", line 56, in message_loop
    msg = self._read_mes

In [ ]:
unmount_ramdrive(mountpoint)

### Predictions

In [51]:
root

PosixPath('/mnt/dataset/shrec-2025-protein-classification/v2-20250331')

In [52]:
test_df = pd.read_csv(root / 'test_set.csv'); test_df.head()

,anonymised_protein_id
0,0.vtk
1,1.vtk
2,2.vtk
3,3.vtk
4,4.vtk


In [63]:
class Predictor():
    def __init__(self, df, model, max_points, root=root, extention='vtk'):
        self.df = df
        model.eval
        self.model = model
        self.max_points = max_points
        self.root = root
        self.extention = extention
        
        self.normalizer = UnitSphereNormalization()
        self.sampler = RandomSampler(sample_size=max_points)

        if self.extention == 'vtk':
            self.root = self.root / 'test'
        
        else:
            raise NotImplementedError()
        
        self.test_run = 0 
        
    def predict(self, get_logits=False):
        prediction_column_name = f'Run_{self.test_run}'
        logits_column_name = f'Logits_{prediction_column_name}'

        pred_col = []
        logit_col = []

        for idx in tqdm(range(len(self.df))):
            protein_id = self.df['anonymised_protein_id'].loc[idx]

            point_cloud = self.get_vtk_points(protein_id)

            point_cloud_t = torch.from_numpy(point_cloud).to(torch.device('cuda:0'))     
            point_cloud_t = point_cloud_t.type(torch.float32)

            point_cloud_t = self.normalizer(point_cloud_t)
            point_cloud_t = self.sampler(point_cloud_t)
            point_cloud_t = torch.transpose(point_cloud_t, 0, 1)

            prediction_logits = self.model(point_cloud_t.view(1, *point_cloud_t.shape))
            prediction = torch.argmax(prediction_logits, dim=1)

            pred_col.append(int(prediction.cpu()))
            logit_col.append(prediction_logits.cpu().detach().numpy())
        
        self.df[prediction_column_name] = pred_col
        if get_logits:
            self.df[logits_column_name] = logit_col

        self.test_run += 1            

    def get_vtk_points(self, name):
        prot_file = name
        prot_file = os.path.join(self.root, prot_file)
        
        prot_mesh = pv.read(prot_file)

        return prot_mesh.points
    


In [54]:
model = PointNet(97, max_points=4000).to(torch.device('cuda:0'))
path_to_weights = Path('/home/giorgio/venvs/SHREC/notebooks/output/run_shrec-2025_08042025_122841_train_set_comprehensive/model_final.pth')
model.load_state_dict(torch.load(path_to_weights, map_location=torch.device('cuda:0')))

Using PointNet as encoder - encoder: PointNetEncoder(
  (input_transform): TNet(
    (shared_mlps): Sequential(
      (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (1): ReLU()
      (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (3): ReLU()
      (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (5): ReLU()
    )
    (linear): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=9, bias=True)
    )
  )
  (feature_transform): TNet(
    (shared_mlps): Sequential(
      (0): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (1): ReLU()
      (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (3): ReLU()
      (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (5): ReLU()
    )
    (linear): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
     

<All keys matched successfully>

In [ ]:
predi = Predictor(test_df, model, max_points=4000)
predi.predict()

 24%|██▍       | 554/2321 [01:23<03:54,  7.52it/s]